# MAVIS Astrometric Simulator

## Phase A Version
EtE Version

## Todo
1. Run MAVISIM with 11x11 sources, equally bright at the coordinates of the PSFs. Get DAOPhot to measure their positions and plot the position error as a vector plot at the coordinates of the sources.
2. Do the exact same but using only the centre PSF (static psf) but still 11x11 sources.

# Imports

In [ ]:
# Standard
import numpy as np
import matplotlib.pyplot as plt

# Astropy
from astropy.io import fits, ascii

import input_parameters_v1_1 as input_par

from mavisim.generate_image import ImageGenerator

from mavisim.source import Source
from mavisim.input_coo import InputCoo
from mavisim.noise import add_all_noise

# Store the save path and file name

In [ ]:
file_name = "PST_StaticPSF45FoV"
image_save_path = "./TestImages/LongInt/"
coo_save_path = "./TestCoordinates/LongInt/"

# Exposure time in seconds
exp_time = 30

# Create the source object (to populate the image)

In [ ]:
source = Source(input_par, exp_time, static_dist=True)
source.build_source()

# Create the input catalogue to compare with DAOPhot

In [ ]:
input_coo = InputCoo(input_par, source).main()

# Create the Noise-Free Image with E2E PSF

## Initialise TileGenerator object + do FFTs of PSFs

# NOTES to try fixing offset

1. DAOPhot Problem? Try changing the FWHM of the Gaussian, increase the size of the vibration or CD kernel
    1. Jesse's PSFs FWHM on the order 15 mas, before CD (FWHM=6.9 mas) and vibration (FWHM~8 mas) errors
    - At centre of FoV get 15.6 mas
    
    2. Try using PSF at longer wavelength?
    
2. Tiling Problem? Try using existing tiling procedure and then playing with Gaussian offset

3. Is there a correlation with the position of the star in the image

4. Gaussian offset set to 0, set positions to integer pixel positions
    1. Try pre-processing source list, divide by 3.75e-3 round down or up then multiply back


In [ ]:
image_gen = ImageGenerator(array_width_pix=12800, pixsize=3.75e-3, source=source,
                            psfs_file="datav1_1/e2e_psfs_100s_lqg_optsquare.fits", gauss_width_pix=34)

image_gen.main()

# Can be rebinned to 30"±3n"
image_final = image_gen.get_rebinned_cropped(rebin_factor=2,cropped_width_as=30.0)

In [ ]:
print (image_final.shape)

In [ ]:
# plot final image:
extent = 42*np.array([-0.5,0.5,-0.5,0.5])
plt.figure(figsize=[10,10])
plt.imshow(image_final, extent=extent)
plt.colorbar()
plt.clim([8,15])

# Add all noise

In [ ]:
# Add sky background,
#shot noise, read noise and convert from electrons to ADU
image_allnoise = add_all_noise(input_par, image_final, source.exp_time)

# Save the final image and input catalogue

In [ ]:
image_final_noise = np.array(image_allnoise, dtype="float32")
hdu = fits.PrimaryHDU(image_final_noise)
hdul = fits.HDUList([hdu])
hdul.writeto("Test1_1.fits", overwrite=True)


In [ ]:
ascii.write(input_coo, 'Test1_1.all', overwrite=True)

# Experiment with rebinning vs not

In [ ]:
norb_image = fits.open(image_save_path + "mavisim2test_JCFixMarchNoRBBigVibNoNoise.fits")
rb_image = fits.open(image_save_path + "mavisim2test_JCFixMarchRBBigVibNoNoise.fits")

dat_norb = norb_image[0].data
slice_norb = dat_norb[3900:4100, 3900:4100]

dat_rb = rb_image[0].data
slice_rb = dat_rb[1900:2100, 1900:2100]


print (np.where(slice_norb == np.amax(slice_norb)))
print (np.where(slice_rb == np.amax(slice_rb)))


print (np.sum(slice_rb), np.sum(slice_norb), source["Flux"], source["Gauss_Info"][0])
print (np.amax(slice_rb), np.amax(slice_norb))


image = np.array(slice_rb, dtype="float32")
hdu = fits.PrimaryHDU(image)
hdul = fits.HDUList([hdu])
hdul.writeto(image_save_path + "sliceRB.fits", overwrite=True)

image = np.array(slice_norb, dtype="float32")
hdu = fits.PrimaryHDU(image)
hdul = fits.HDUList([hdu])
hdul.writeto(image_save_path + "sliceNoRB.fits", overwrite=True)
